In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from models.vanillann import YieldDataset, SimpleModel

%load_ext autoreload
%autoreload 2

### Specify use case through crop and country

In [2]:
# USER INPUTS
country = "BR" # one of ["US", "BR"]
crop = "wheat" # one of ["maize", "wheat"]

match country:
    case "US":
        match crop:
            case "wheat":
                ecmwf_path = "data/preprocessed/US/ecmwf_era_wheat_US.csv"
                predictor_path = "data/preprocessed/US/ndvi_soil_soil_moisture_wheat_US.csv"
                yield_path = "data/CY-Bench/US/wheat/yield_wheat_US.csv"
                test_years = [2020, 2021, 2022]
                print(country, crop, ecmwf_path, predictor_path, yield_path, test_years)
            case "maize":
                ecmwf_path = "data/preprocessed/US/ecmwf_era_maize_US.csv"
                predictor_path = "data/preprocessed/US/ndvi_soil_soil_moisture_maize_US.csv"
                yield_path = "data/CY-Bench/US/maize/yield_maize_US.csv"
                test_years = [2020, 2021, 2022]
                print(country, crop, ecmwf_path, predictor_path, yield_path, test_years)
            case _:
                print("invalid crop, has to be wheat or maize")
    case "BR":
        match crop:
            case "wheat":
                ecmwf_path = "data/preprocessed/BR/ecmwf_era_wheat_BR.csv"
                predictor_path = "data/preprocessed/BR/ndvi_soil_soil_moisture_wheat_BR.csv"
                yield_path = "data/CY-Bench/BR/wheat/yield_wheat_BR.csv"
                test_years = [2006, 2015, 2017]
                print(country, crop, ecmwf_path, predictor_path, yield_path, test_years)
            case "maize":
                ecmwf_path = "data/preprocessed/BR/ecmwf_era_maize_BR.csv"
                predictor_path = "data/preprocessed/BR/ndvi_soil_soil_moisture_maize_BR.csv"
                yield_path = "data/CY-Bench/BR/maize/yield_maize_BR.csv"
                test_years = [2020, 2021, 2022]
                print(country, crop, ecmwf_path, predictor_path, yield_path, test_years)
            case _:
                print("invalid crop, has to be wheat or maize")
    case _:
        print("invalid country, has to be US or BR")


BR wheat data/preprocessed/BR/ecmwf_era_wheat_BR.csv data/preprocessed/BR/ndvi_soil_soil_moisture_wheat_BR.csv data/CY-Bench/BR/wheat/yield_wheat_BR.csv [2006, 2015, 2017]


### Read yield and SCM_ERA data

In [3]:
# Historical yield data
y = pd.read_csv(yield_path)
y = y.loc[y["harvest_year"].between(2003, 2023), ["adm_id", "harvest_year", "yield", "harvested_area"]].reset_index(drop=True)

# Merge predictor data
x_1 = pd.read_csv(ecmwf_path)
x_2 = pd.read_csv(predictor_path)
x = x_1.merge(x_2, on=["adm_id", "harvest_year"], how="left").dropna().reset_index(drop=True)

# Merge predictor and yield data
x_y = x.merge(y, on=["adm_id", "harvest_year"], how="inner")

# remove test years
train_df = x_y[~x_y['harvest_year'].isin(test_years)].reset_index(drop=True)
train_df.head()

,adm_id,init_date,tavg_16,tavg_17,tavg_18,tavg_19,tavg_20,tavg_21,tavg_22,tavg_23,...,ndvi_37,ndvi_38,ndvi_39,ndvi_40,ndvi_41,awc,bulk_density,drainage_class,yield,harvested_area
0,BR2311504,2020-05-01,25.653409,25.939799,26.185234,26.433421,26.679016,26.861418,26.954018,26.953981,...,121.718919,125.009244,128.299569,126.590551,125.689655,13.027722,1.443607,4.0,5.4,5.0
1,BR2311504,2020-06-01,26.782667,26.057738,26.166207,26.364367,25.261646,26.209929,26.481371,26.627659,...,121.718919,125.009244,128.299569,126.590551,125.689655,13.027722,1.443607,4.0,5.4,5.0
2,BR2311504,2020-07-01,26.782667,26.057738,26.166207,26.364367,25.930489,26.029086,26.366901,26.227341,...,121.718919,125.009244,128.299569,126.590551,125.689655,13.027722,1.443607,4.0,5.4,5.0
3,BR2311504,2020-08-01,26.782667,26.057738,26.166207,26.364367,25.930489,26.029086,26.366901,25.875049,...,121.718919,125.009244,128.299569,126.590551,125.689655,13.027722,1.443607,4.0,5.4,5.0
4,BR2311504,2020-09-01,26.782667,26.057738,26.166207,26.364367,25.930489,26.029086,26.366901,25.875049,...,121.718919,125.009244,128.299569,126.590551,125.689655,13.027722,1.443607,4.0,5.4,5.0


In [8]:
south_of_brazil = ['BR4100103','BR4100202','BR4100301','BR4100400','BR4100459','BR4100509','BR4100608','BR4100707','BR4100806','BR4100905','BR4101002','BR4101051','BR4101101','BR4101150','BR4101200','BR4101309','BR4101408','BR4101507','BR4101606','BR4101655','BR4101705','BR4101804','BR4101853','BR4101903','BR4102000','BR4102109','BR4102208','BR4102307','BR4102406','BR4102505','BR4102604','BR4102703','BR4102752','BR4102802','BR4102901','BR4103008','BR4103024','BR4103040','BR4103057','BR4103107','BR4103156','BR4103206','BR4103222','BR4103305','BR4103354','BR4103370','BR4103404','BR4103453','BR4103479','BR4103503','BR4103602','BR4103701','BR4103800','BR4103909','BR4103958','BR4104006','BR4104055','BR4104105','BR4104204','BR4104253','BR4104303','BR4104402','BR4104428','BR4104451','BR4104501','BR4104600','BR4104659','BR4104709','BR4104808','BR4104907','BR4105003','BR4105102','BR4105201','BR4105300','BR4105409','BR4105508','BR4105607','BR4105706','BR4105805','BR4105904','BR4106001','BR4106100','BR4106209','BR4106308','BR4106407','BR4106456','BR4106506','BR4106555','BR4106571','BR4106605','BR4106704','BR4106803','BR4106852','BR4106902','BR4107009','BR4107108','BR4107124','BR4107157','BR4107207','BR4107256','BR4107306','BR4107405','BR4107504','BR4107520','BR4107538','BR4107546','BR4107553','BR4107603','BR4107652','BR4107702','BR4107736','BR4107751','BR4107801','BR4107850','BR4107900','BR4108007','BR4108106','BR4108205','BR4108304','BR4108320','BR4108403','BR4108452','BR4108502','BR4108551','BR4108601','BR4108650','BR4108700','BR4108809','BR4108908','BR4108957','BR4109005','BR4109104','BR4109203','BR4109302','BR4109401','BR4109500','BR4109609','BR4109658','BR4109708','BR4109757','BR4109807','BR4109906','BR4110003','BR4110052','BR4110078','BR4110102','BR4110201','BR4110300','BR4110409','BR4110508','BR4110607','BR4110656','BR4110706','BR4110805','BR4110904','BR4110953','BR4111001','BR4111100','BR4111209','BR4111258','BR4111308','BR4111407','BR4111506','BR4111555','BR4111605','BR4111704','BR4111803','BR4111902','BR4112009','BR4112108','BR4112207','BR4112306','BR4112405','BR4112504','BR4112603','BR4112702','BR4112751','BR4112801','BR4112900','BR4112959','BR4113007','BR4113106','BR4113205','BR4113254','BR4113304','BR4113403','BR4113429','BR4113452','BR4113502','BR4113601','BR4113700','BR4113734','BR4113759','BR4113809','BR4113908','BR4114005','BR4114104','BR4114203','BR4114302','BR4114351','BR4114401','BR4114500','BR4114609','BR4114708','BR4114807','BR4114906','BR4115002','BR4115101','BR4115200','BR4115309','BR4115358','BR4115408','BR4115457','BR4115507','BR4115606','BR4115705','BR4115739','BR4115754','BR4115804','BR4115853','BR4115903','BR4116000','BR4116059','BR4116109','BR4116208','BR4116307','BR4116406','BR4116505','BR4116604','BR4116703','BR4116802','BR4116901','BR4116950','BR4117008','BR4117057','BR4117107','BR4117206','BR4117214','BR4117222','BR4117255','BR4117271','BR4117297','BR4117305','BR4117404','BR4117453','BR4117503','BR4117602','BR4117701','BR4117800','BR4117909','BR4118006','BR4118105','BR4118204','BR4118303','BR4118402','BR4118451','BR4118501','BR4118600','BR4118709','BR4118808','BR4118857','BR4118907','BR4119004','BR4119103','BR4119152','BR4119202','BR4119251','BR4119301','BR4119400','BR4119509','BR4119608','BR4119657','BR4119707','BR4119806','BR4119905','BR4119954','BR4120002','BR4120101','BR4120150','BR4120200','BR4120309','BR4120333','BR4120358','BR4120408','BR4120507','BR4120606','BR4120655','BR4120705','BR4120804','BR4120853','BR4120903','BR4121000','BR4121109','BR4121208','BR4121257','BR4121307','BR4121356','BR4121406','BR4121505','BR4121604','BR4121703','BR4121752','BR4121802','BR4121901','BR4122008','BR4122107','BR4122156','BR4122172','BR4122206','BR4122305','BR4122404','BR4122503','BR4122602','BR4122651','BR4122701','BR4122800','BR4122909','BR4123006','BR4123105','BR4123204','BR4123303','BR4123402','BR4123501','BR4123600','BR4123709','BR4123808','BR4123824','BR4123857','BR4123907','BR4123956','BR4124004','BR4124020','BR4124053','BR4124103','BR4124202','BR4124301','BR4124400','BR4124509','BR4124608','BR4124707','BR4124806','BR4124905','BR4125001','BR4125100','BR4125209','BR4125308','BR4125357','BR4125407','BR4125456','BR4125506','BR4125555','BR4125605','BR4125704','BR4125753','BR4125803','BR4125902','BR4126009','BR4126108','BR4126207','BR4126256','BR4126272','BR4126306','BR4126355','BR4126405','BR4126504','BR4126603','BR4126652','BR4126678','BR4126702','BR4126801','BR4126900','BR4127007','BR4127106','BR4127205','BR4127304','BR4127403','BR4127502','BR4127601','BR4127700','BR4127809','BR4127858','BR4127882','BR4127908','BR4127957','BR4127965','BR4128005','BR4128104','BR4128203','BR4128302','BR4128401','BR4128500','BR4128534','BR4128559','BR4128609','BR4128625','BR4128633','BR4128658','BR4128708','BR4128807','BR4200051','BR4200101','BR4200200','BR4200309','BR4200408','BR4200507','BR4200556','BR4200606','BR4200705','BR4200754','BR4200804','BR4200903','BR4201000','BR4201109','BR4201208','BR4201257','BR4201273','BR4201307','BR4201406','BR4201505','BR4201604','BR4201653','BR4201703','BR4201802','BR4201901','BR4201950','BR4202008','BR4202057','BR4202073','BR4202081','BR4202099','BR4202107','BR4202131','BR4202156','BR4202206','BR4202305','BR4202404','BR4202438','BR4202453','BR4202503','BR4202537','BR4202578','BR4202602','BR4202701','BR4202800','BR4202859','BR4202875','BR4202909','BR4203006','BR4203105','BR4203154','BR4203204','BR4203253','BR4203303','BR4203402','BR4203501','BR4203600','BR4203709','BR4203808','BR4203907','BR4203956','BR4204004','BR4204103','BR4204152','BR4204178','BR4204194','BR4204202','BR4204251','BR4204301','BR4204350','BR4204400','BR4204459','BR4204509','BR4204558','BR4204608','BR4204707','BR4204756','BR4204806','BR4204905','BR4205001','BR4205100','BR4205159','BR4205175','BR4205191','BR4205209','BR4205308','BR4205357','BR4205407','BR4205431','BR4205456','BR4205506','BR4205555','BR4205605','BR4205704','BR4205803','BR4205902','BR4206009','BR4206108','BR4206207','BR4206306','BR4206405','BR4206504','BR4206603','BR4206652','BR4206702','BR4206751','BR4206801','BR4206900','BR4207007','BR4207106','BR4207205','BR4207304','BR4207403','BR4207502','BR4207577','BR4207601','BR4207650','BR4207684','BR4207700','BR4207759','BR4207809','BR4207858','BR4207908','BR4208005','BR4208104','BR4208203','BR4208302','BR4208401','BR4208450','BR4208500','BR4208609','BR4208708','BR4208807','BR4208906','BR4208955','BR4209003','BR4209102','BR4209151','BR4209177','BR4209201','BR4209300','BR4209409','BR4209458','BR4209508','BR4209607','BR4209706','BR4209805','BR4209854','BR4209904','BR4210001','BR4210035','BR4210050','BR4210100','BR4210209','BR4210308','BR4210407','BR4210506','BR4210555','BR4210605','BR4210704','BR4210803','BR4210852','BR4210902','BR4211009','BR4211058','BR4211108','BR4211207','BR4211256','BR4211306','BR4211405','BR4211454','BR4211504','BR4211603','BR4211652','BR4211702','BR4211751','BR4211801','BR4211850','BR4211876','BR4211892','BR4211900','BR4212007','BR4212056','BR4212106','BR4212205','BR4212239','BR4212254','BR4212270','BR4212304','BR4212403','BR4212502','BR4212601','BR4212650','BR4212700','BR4212809','BR4212908','BR4213005','BR4213104','BR4213153','BR4213203','BR4213302','BR4213351','BR4213401','BR4213500','BR4213609','BR4213708','BR4213807','BR4213906','BR4214003','BR4214102','BR4214151','BR4214201','BR4214300','BR4214409','BR4214508','BR4214607','BR4214706','BR4214805','BR4214904','BR4215000','BR4215059','BR4215075','BR4215109','BR4215208','BR4215307','BR4215356','BR4215406','BR4215455','BR4215505','BR4215554','BR4215604','BR4215653','BR4215679','BR4215687','BR4215695','BR4215703','BR4215752','BR4215802','BR4215901','BR4216008','BR4216057','BR4216107','BR4216206','BR4216255','BR4216305','BR4216354','BR4216404','BR4216503','BR4216602','BR4216701','BR4216800','BR4216909','BR4217006','BR4217105','BR4217154','BR4217204','BR4217253','BR4217303','BR4217402','BR4217501','BR4217550','BR4217600','BR4217709','BR4217758','BR4217808','BR4217907','BR4217956','BR4218004','BR4218103','BR4218202','BR4218251','BR4218301','BR4218350','BR4218400','BR4218509','BR4218608','BR4218707','BR4218756','BR4218806','BR4218855','BR4218905','BR4218954','BR4219002','BR4219101','BR4219150','BR4219176','BR4219200','BR4219309','BR4219358','BR4219408','BR4219507','BR4219606','BR4219705','BR4219853','BR4220000','BR4300001','BR4300002','BR4300034','BR4300059','BR4300109','BR4300208','BR4300307','BR4300406','BR4300455','BR4300471','BR4300505','BR4300554','BR4300570','BR4300604','BR4300638','BR4300646','BR4300661','BR4300703','BR4300802','BR4300851','BR4300877','BR4300901','BR4301008','BR4301057','BR4301073','BR4301107','BR4301206','BR4301305','BR4301404','BR4301503','BR4301552','BR4301602','BR4301636','BR4301651','BR4301701','BR4301750','BR4301800','BR4301859','BR4301875','BR4301909','BR4301925','BR4301958','BR4302006','BR4302055','BR4302105','BR4302154','BR4302204','BR4302220','BR4302238','BR4302253','BR4302303','BR4302352','BR4302378','BR4302402','BR4302451','BR4302501','BR4302584','BR4302600','BR4302659','BR4302709','BR4302808','BR4302907','BR4303004','BR4303103','BR4303202','BR4303301','BR4303400','BR4303509','BR4303558','BR4303608','BR4303673','BR4303707','BR4303806','BR4303905','BR4304002','BR4304101','BR4304200','BR4304309','BR4304358','BR4304408','BR4304507','BR4304606','BR4304614','BR4304622','BR4304630','BR4304655','BR4304663','BR4304671','BR4304689','BR4304697','BR4304705','BR4304713','BR4304804','BR4304853','BR4304903','BR4304952','BR4305009','BR4305108','BR4305116','BR4305124','BR4305132','BR4305157','BR4305173','BR4305207','BR4305306','BR4305355','BR4305371','BR4305405','BR4305439','BR4305447','BR4305454','BR4305504','BR4305587','BR4305603','BR4305702','BR4305801','BR4305835','BR4305850','BR4305871','BR4305900','BR4305934','BR4305959','BR4305975','BR4306007','BR4306056','BR4306072','BR4306106','BR4306130','BR4306205','BR4306304','BR4306320','BR4306353','BR4306379','BR4306403','BR4306429','BR4306452','BR4306502','BR4306551','BR4306601','BR4306700','BR4306734','BR4306759','BR4306767','BR4306809','BR4306908','BR4306924','BR4306932','BR4306957','BR4306973','BR4307005','BR4307054','BR4307104','BR4307203','BR4307302','BR4307401','BR4307450','BR4307500','BR4307559','BR4307609','BR4307708','BR4307807','BR4307815','BR4307831','BR4307864','BR4307906','BR4308003','BR4308052','BR4308078','BR4308102','BR4308201','BR4308250','BR4308300','BR4308409','BR4308433','BR4308458','BR4308508','BR4308607','BR4308656','BR4308706','BR4308805','BR4308854','BR4308904','BR4309001','BR4309050','BR4309100','BR4309126','BR4309159','BR4309209','BR4309258','BR4309308','BR4309407','BR4309506','BR4309555','BR4309571','BR4309605','BR4309654','BR4309704','BR4309753','BR4309803','BR4309902','BR4309951','BR4310009','BR4310108','BR4310207','BR4310306','BR4310330','BR4310363','BR4310405','BR4310413','BR4310439','BR4310462','BR4310504','BR4310538','BR4310553','BR4310579','BR4310603','BR4310652','BR4310702','BR4310751','BR4310801','BR4310850','BR4310876','BR4310900','BR4311007','BR4311106','BR4311122','BR4311130','BR4311155','BR4311205','BR4311239','BR4311254','BR4311270','BR4311304','BR4311403','BR4311429','BR4311502','BR4311601','BR4311627','BR4311643','BR4311700','BR4311718','BR4311734','BR4311759','BR4311775','BR4311791','BR4311809','BR4311908','BR4311981','BR4312005','BR4312054','BR4312104','BR4312138','BR4312153','BR4312179','BR4312203','BR4312252','BR4312302','BR4312351','BR4312377','BR4312385','BR4312401','BR4312427','BR4312443','BR4312450','BR4312476','BR4312500','BR4312609','BR4312617','BR4312625','BR4312658','BR4312674','BR4312708','BR4312757','BR4312807','BR4312906','BR4312955','BR4313003','BR4313011','BR4313037','BR4313060','BR4313086','BR4313102','BR4313201','BR4313300','BR4313334','BR4313359','BR4313375','BR4313391','BR4313409','BR4313425','BR4313441','BR4313466','BR4313490','BR4313508','BR4313607','BR4313656','BR4313706','BR4313805','BR4313904','BR4313953','BR4314001','BR4314027','BR4314035','BR4314050','BR4314068','BR4314076','BR4314100','BR4314134','BR4314159','BR4314175','BR4314209','BR4314308','BR4314407','BR4314423','BR4314456','BR4314464','BR4314472','BR4314498','BR4314506','BR4314548','BR4314555','BR4314605','BR4314704','BR4314753','BR4314779','BR4314787','BR4314803','BR4314902','BR4315008','BR4315057','BR4315073','BR4315107','BR4315131','BR4315149','BR4315156','BR4315172','BR4315206','BR4315305','BR4315313','BR4315321','BR4315354','BR4315404','BR4315453','BR4315503','BR4315552','BR4315602','BR4315701','BR4315750','BR4315800','BR4315909','BR4315958','BR4316006','BR4316105','BR4316204','BR4316303','BR4316402','BR4316428','BR4316436','BR4316451','BR4316477','BR4316501','BR4316600','BR4316709','BR4316733','BR4316758','BR4316808','BR4316907','BR4316956','BR4316972','BR4317004','BR4317103','BR4317202','BR4317251','BR4317301','BR4317400','BR4317509','BR4317558','BR4317608','BR4317707','BR4317756','BR4317806','BR4317905','BR4317954','BR4318002','BR4318051','BR4318101','BR4318200','BR4318309','BR4318408','BR4318424','BR4318432','BR4318440','BR4318457','BR4318465','BR4318481','BR4318499','BR4318507','BR4318606','BR4318614','BR4318622','BR4318705','BR4318804','BR4318903','BR4319000','BR4319109','BR4319125','BR4319158','BR4319208','BR4319307','BR4319356','BR4319364','BR4319372','BR4319406','BR4319505','BR4319604','BR4319703','BR4319711','BR4319737','BR4319752','BR4319802','BR4319901','BR4320008','BR4320107','BR4320206','BR4320230','BR4320263','BR4320305','BR4320321','BR4320354','BR4320404','BR4320453','BR4320503','BR4320552','BR4320578','BR4320602','BR4320651','BR4320677','BR4320701','BR4320800','BR4320859','BR4320909','BR4321006','BR4321105','BR4321204','BR4321303','BR4321329','BR4321352','BR4321402','BR4321436','BR4321451','BR4321469','BR4321477','BR4321493','BR4321501','BR4321600','BR4321626','BR4321634','BR4321667','BR4321709','BR4321808','BR4321832','BR4321857','BR4321907','BR4321956','BR4322004','BR4322103','BR4322152','BR4322186','BR4322202','BR4322251','BR4322301','BR4322327','BR4322343','BR4322350','BR4322376','BR4322400','BR4322509','BR4322525','BR4322533','BR4322541','BR4322558','BR4322608','BR4322707','BR4322806','BR4322855','BR4322905','BR4323002','BR4323101','BR4323200','BR4323309','BR4323358','BR4323408','BR4323457','BR4323507','BR4323606','BR4323705','BR4323754','BR4323770','BR4323804']

In [4]:
no_feature = ["adm_id", "init_date", "harvest_year", "yield", "harvested_area"]

In [12]:
train_df = train_df.loc[train_df["adm_id"].isin(south_of_brazil), ].reset_index(drop=True)

In [13]:
end_of_season_df = train_df[pd.to_datetime(train_df["init_date"]).dt.month == 12].drop(columns=["init_date", "harvested_area"]).set_index("adm_id")

In [14]:
def RMSELoss(yhat,y):
    return 100 * torch.sqrt(torch.mean((yhat-y)**2)) / torch.mean(y)

In [17]:
unique_years = end_of_season_df['harvest_year'].unique()
unique_years.sort()
results = dict.fromkeys(unique_years)

batch_size = 64

for year in unique_years:
    print(f'Validating on year {year}')
    
    # Create training and validation sets for this fold
    train_fold_df = end_of_season_df[end_of_season_df['harvest_year'] != year]
    val_fold_df = end_of_season_df[end_of_season_df['harvest_year'] == year]
    
    train_fold_features = train_fold_df[[c for c in train_fold_df.columns if c not in no_feature]]
    train_fold_target = train_fold_df['yield']
    val_fold_features = val_fold_df[[c for c in val_fold_df.columns if c not in no_feature]]
    val_fold_target = val_fold_df['yield']
    
    means = train_fold_features.mean()
    stds = train_fold_features.std()
    train_fold_features = (train_fold_features - means) / stds
    val_fold_features = (val_fold_features - means) / stds
    
    train_fold_dataset = YieldDataset(train_fold_features, train_fold_target)
    val_fold_dataset = YieldDataset(val_fold_features, val_fold_target)
    
    train_fold_loader = DataLoader(train_fold_dataset, batch_size=batch_size, shuffle=True)
    val_fold_loader = DataLoader(val_fold_dataset, batch_size=batch_size, shuffle=False)
    
    # Reset the model and optimizer
    model = SimpleModel()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = RMSELoss
    
    # Early stopping parameters
    num_epochs = 10  # Set the maximum number of epochs you want to train for
    patience = 4  # Number of epochs to wait for improvement before stopping
    best_val_loss = float('inf')  # Initialize the best validation loss
    epochs_no_improve = 0  # Counter for epochs without improvement
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for features, target in train_fold_loader:
            optimizer.zero_grad()
            output = model(features)
            loss = criterion(output, target.unsqueeze(1))
            loss.backward()
            optimizer.step()
        
        # Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for features, target in val_fold_loader:
                output = model(features)
                loss = criterion(output, target.unsqueeze(1))
                val_loss += loss.item()
        
        val_loss /= len(val_fold_loader)  # Compute the average validation loss
        print(f'Epoch {epoch + 1}, Validation Loss for year {year}: {val_loss}')
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0  # Reset the counter if validation loss improves
        else:
            epochs_no_improve += 1  # Increment the counter if validation loss does not improve
        
        if epochs_no_improve >= patience:
            print(f'Early stopping at epoch {epoch + 1}')
            break  # Stop training if no improvement for specified number of epochs
    
    results[year] = best_val_loss
        
# Once cross-validation is done, you can test on the test dataset using test_loader


Validating on year 2003
Epoch 1, Validation Loss for year 2003: 28.83928401653583
Epoch 2, Validation Loss for year 2003: 27.88056623018705
Epoch 3, Validation Loss for year 2003: 25.97467305110051
Epoch 4, Validation Loss for year 2003: 27.600949654212364
Epoch 5, Validation Loss for year 2003: 26.73481134267954
Epoch 6, Validation Loss for year 2003: 24.84883367098295
Epoch 7, Validation Loss for year 2003: 24.98468780517578
Epoch 8, Validation Loss for year 2003: 23.951876420241135
Epoch 9, Validation Loss for year 2003: 24.096622467041016
Epoch 10, Validation Loss for year 2003: 23.29017917926495
Validating on year 2004
Epoch 1, Validation Loss for year 2004: 34.171176365443635
Epoch 2, Validation Loss for year 2004: 34.728364263262065
Epoch 3, Validation Loss for year 2004: 37.611723218645366
Epoch 4, Validation Loss for year 2004: 37.14175346919468
Epoch 5, Validation Loss for year 2004: 36.46329321180071
Early stopping at epoch 5
Validating on year 2005
Epoch 1, Validation Loss 

KeyboardInterrupt: 